# 3D gamma from DICOM
## example for a patient orientation of "head first decubitus left"
PyMedPhys has multiple ways to calculate gamma. There are also a range of interfaces that can be used. Presented here is a simplified interface which receives as its input two DICOM file paths for the purpose of directly calculating 3-dimensional gamma from a pair of RT DICOM dose files.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
!pip install pydicom pymedphys

import pydicom
import pymedphys

You should consider upgrading via the 'd:\python\python37\python.exe -m pip install --upgrade pip' command.


## Define common patient scan orientations

In [3]:
Patient_Orientation = []
Patient_Orientation.append(["Head first supine", [1, 0, 0, 0, 1, 0], 'A', 'P', 'R', 'L']) #patient orientation description, value of tag (0020,0037), orientation on a slice 
Patient_Orientation.append(["Head first prone", [-1, 0, 0, 0, -1, 0], 'P', 'A', 'L', 'R'])
Patient_Orientation.append(["Head first decubitus right", [0, 1, 0, -1, 0, 0], 'L', 'R', 'A', 'P'])
Patient_Orientation.append(["Head first decubitus left", [0, -1, 0, 1, 0, 0], 'R', 'L', 'P', 'A'])
Patient_Orientation.append(["Feet first supine", [-1, 0, 0, 0, 1, 0], 'A', 'P', 'L', 'R'])
Patient_Orientation.append(["Feet first prone", [1, 0, 0, 0, -1, 0], 'P', 'A', 'R', 'L'])
Patient_Orientation.append(["Feet first decubitus right", [0, -1, 0, -1, 0, 0], 'L', 'R', 'P', 'A'])
Patient_Orientation.append(["Feet first decubitus left", [0, 1, 0, 1, 0, 0], 'R', 'L', 'A', 'P'])

## Getting two demo DICOM dose files

Let's download some demo files for the purpose of demonstrating `gamma_dicom` usage.

In [4]:
reference_filepath = pymedphys.data_path("RD.HFDL_ref.dcm")
reference_filepath

ValueError: The file provided isn't within pymedphys' urls.json record.

In [ ]:
evaluation_filepath = pymedphys.data_path("RD.HFDL_eva.dcm")
evaluation_filepath

In [ ]:
reference = pydicom.read_file(str(reference_filepath), force=True)
evaluation = pydicom.read_file(str(evaluation_filepath), force=True)

In [ ]:
pt_axes_ref, dose_reference = pymedphys.dicom.zyx_and_dose_from_dataset(reference)
pt_axes_eva, dose_evaluation = pymedphys.dicom.zyx_and_dose_from_dataset(evaluation)

In [ ]:
pt_orient_ref = reference[0x20,0x37].value # tag of Image Orientation (Patient) is 0020,0037

In [ ]:
A = [] # define A as the translation matrix
A.append(pt_orient_ref[0:3]) #cosine between DICOM x axis and patient x, y, and z axis
A.append(pt_orient_ref[3:6]) #cosine between DICOM y axis and patient x, y, and z axis
A.append([pt_orient_ref[1]*pt_orient_ref[5]-pt_orient_ref[2]*pt_orient_ref[4],
          pt_orient_ref[2]*pt_orient_ref[3]-pt_orient_ref[0]*pt_orient_ref[5],
          pt_orient_ref[0]*pt_orient_ref[4]-pt_orient_ref[1]*pt_orient_ref[3]])
          #cosine between DICOM z axis and patient x, y, and z axis
print('Translation matrix from patient coordinate system to DICOM coordinate system: {0}\n'.format(A))

## Translate patient coordinate system to DICOM coordinate system

In [ ]:
#np.nonzero returns a tuple, so [0][0] is needed to get the index of first non zero value in a list
i_z_dcm = np.nonzero(A[2])[0][0] #index for z_axis in DICOM
i_y_dcm = np.nonzero(A[1])[0][0] #index for y_axis in DICOM
i_x_dcm = np.nonzero(A[0])[0][0] #index for x_axis in DICOM

for i in range(0, len(Patient_Orientation)):
    if pt_orient_ref == Patient_Orientation[i][1]:
        calculationLog.write('Patient scan orientation: {0}\n'.format(Patient_Orientation[i][0]))
        current_pt_orient = Patient_Orientation[i][0] # string of the patient scan orientation
        current_pt_orient_marks = Patient_Orientation[i][2:6] # four letters to indicate patient orientation on slice
        #print('Patient scan orientation: {0}\n'.format(current_pt_orient))
        #print('Patient scan orientation marks on a slice: {0}\n'.format(current_pt_orient_marks))
        axes_reference = (pt_axes_ref[2-i_z_dcm]*A[2][i_z_dcm], pt_axes_ref[2-i_y_dcm]*A[1][i_y_dcm], pt_axes_ref[2-i_x_dcm]*A[0][i_x_dcm]) #DICOM z,y,x axes
        axes_evaluation = (pt_axes_eva[2-i_z_dcm]*A[2][i_z_dcm], pt_axes_eva[2-i_y_dcm]*A[1][i_y_dcm], pt_axes_eva[2-i_x_dcm]*A[0][i_x_dcm])

In [ ]:
(z_ref, y_ref, x_ref) = axes_reference
(z_eval, y_eval, x_eval) = axes_evaluation

Importantly the shape of the coordinates are in the same order as the dose axis order

In [ ]:
np.shape(z_ref)

In [ ]:
np.shape(y_ref)

In [ ]:
np.shape(x_ref)

In [ ]:
np.shape(dose_reference)

## Calculate and display gamma

In [ ]:
gamma_options = {
    'dose_percent_threshold': 1, #(1%/1mm) criteria
    'distance_mm_threshold': 1,
    'lower_percent_dose_cutoff': 10, #10% of maximum dose in reference plan
    'interp_fraction': 10,  # Should be 10 or more for more accurate results
    'max_gamma': 5,
    'random_subset': None,
    'local_gamma': False, # Maximum dose in reference plan is used as normalisation factor in global gamma
    'ram_available': 2**29  # 1/2 GB
}
    
gamma = pymedphys.gamma(
    axes_reference, dose_reference, 
    axes_evaluation, dose_evaluation, 
    **gamma_options)

In [ ]:
valid_gamma = gamma[~np.isnan(gamma)]

num_bins = (
    gamma_options['interp_fraction'] * gamma_options['max_gamma'])
bins = np.linspace(0, gamma_options['max_gamma'], num_bins + 1)

plt.hist(valid_gamma, bins, density=True)
#if density is True, y value is probability density; otherwise, it is count in a bin
plt.xlim([0, gamma_options['max_gamma']])
plt.xlabel('gamma index')
plt.ylabel('probability density')
    
pass_ratio = np.sum(valid_gamma <= 1) / len(valid_gamma)

if gamma_options['local_gamma']:
    gamma_norm_condition = 'Local gamma'
else:
    gamma_norm_condition = 'Global gamma'

plt.title(f"Dose cut: {gamma_options['lower_percent_dose_cutoff']}% | {gamma_norm_condition} ({gamma_options['dose_percent_threshold']}%/{gamma_options['distance_mm_threshold']}mm) | Pass Rate(\u03B3<=1): {pass_ratio*100:.2f}% \n ref pts: {len(z_ref)*len(y_ref)*len(x_ref)} | valid \u03B3 pts: {len(valid_gamma)}")

# plt.savefig('gamma_hist.png', dpi=300)

## Plot dose and gamma on sampled slices

In [ ]:
max_ref_dose = np.max(dose_reference)

lower_dose_cutoff = gamma_options['lower_percent_dose_cutoff'] / 100 * max_ref_dose

relevant_slice = (
    np.max(dose_reference, axis=(1, 2)) > 
    lower_dose_cutoff)
slice_start = np.max([
        np.where(relevant_slice)[0][0], 
        0])
slice_end = np.min([
        np.where(relevant_slice)[0][-1], 
        len(z_ref)])

In [ ]:
delta_z_show = 5
z_vals = z_ref[slice(slice_start, slice_end, delta_z_show)]

eval_slices = [
    dose_evaluation[np.where(z_i == z_eval)[0][0], :, :]
    for z_i in z_vals
]

ref_slices = [
    dose_reference[np.where(z_i == z_ref)[0][0], :, :]
    for z_i in z_vals
]

gamma_slices = [
    gamma[np.where(z_i == z_ref)[0][0], :, :]
    for z_i in z_vals
]

diffs = [
    eval_slice - ref_slice
    for eval_slice, ref_slice 
    in zip(eval_slices, ref_slices)
]

max_diff = np.max(np.abs(diffs))



for i, (eval_slice, ref_slice, diff, gamma_slice) in enumerate(zip(eval_slices, ref_slices, diffs, gamma_slices)):    
    fig, ax = plt.subplots(figsize=(13,10), nrows=2, ncols=2)
   
    fig.suptitle('Slice {0}, z={1:.2f} mm in DICOM'.format(slice_start+i*delta_z_show, z_ref[slice_start+i*delta_z_show]), fontsize=12)
    c00 = ax[0,0].contourf(
        x_eval, y_eval, eval_slice, 100, 
        vmin=0, vmax=max_ref_dose, cmap=plt.get_cmap('rainbow'))
    ax[0,0].set_title("Evaluation")
    fig.colorbar(c00, ax=ax[0,0], label='Dose (Gy)')
    ax[0,0].invert_yaxis()
    ax[0,0].set_xlabel('x (mm)')
    ax[0,0].set_ylabel('y (mm)')
    
    c01 = ax[0,1].contourf(
        x_ref, y_ref, ref_slice, 100, 
        vmin=0, vmax=max_ref_dose, cmap=plt.get_cmap('rainbow'))
    ax[0,1].set_title("Reference")  
    fig.colorbar(c01, ax=ax[0,1], label='Dose (Gy)')
    ax[0,1].invert_yaxis()
    ax[0,1].set_xlabel('x (mm)')
    ax[0,1].set_ylabel('y (mm)')

    c10 = ax[1,0].contourf(
        x_ref, y_ref, diff, 100, 
        vmin=-max_diff, vmax=max_diff, cmap=plt.get_cmap('seismic'))
    ax[1,0].set_title("Dose difference")    
    cbar = fig.colorbar(c10, ax=ax[1,0], label='[Dose Eval] - [Dose Ref] (Gy)')
    cbar.formatter.set_powerlimits((0, 0)) #use scientific notation
    ax[1,0].invert_yaxis()
    ax[1,0].set_xlabel('x (mm)')
    ax[1,0].set_ylabel('y (mm)')
    
    c11 = ax[1,1].contourf(
        x_ref, y_ref, gamma_slice, 100, 
        vmin=0, vmax=gamma_options['max_gamma'], cmap=plt.get_cmap('coolwarm'))
    ax[1,1].set_title(
        f"{gamma_norm_condition} ({gamma_options['dose_percent_threshold']} % / {gamma_options['distance_mm_threshold']} mm)")    
    fig.colorbar(c11, ax=ax[1,1], label='gamma index')
    ax[1,1].invert_yaxis()
    ax[1,1].set_xlabel('x (mm)')
    ax[1,1].set_ylabel('y (mm)')
    
    #add four orientation marks
    len_x = axes_evaluation[2][-1] - axes_evaluation[2][0] #length of x axis
    len_y = axes_evaluation[1][-1] - axes_evaluation[1][0] #lenght of y axis
    ax[0,0].text(axes_evaluation[2][-1] + 0.4 * len_x, axes_evaluation[1][0] - 0.1 * len_y, current_pt_orient_marks[0], fontsize=15, fontweight='bold')
    ax[0,0].text(axes_evaluation[2][0] - 0.2 * len_x, axes_evaluation[1][-1] + 0.1 * len_y, current_pt_orient_marks[2], fontsize=15, fontweight='bold')
    ax[1,0].text(axes_evaluation[2][-1] + 0.4 * len_x, axes_evaluation[1][-1] + 0.2 * len_y, current_pt_orient_marks[1], fontsize=15, fontweight='bold')
    ax[0,1].text(axes_evaluation[2][-1] + 0.4 * len_x, axes_evaluation[1][-1] + 0.1 * len_y, current_pt_orient_marks[3], fontsize=15, fontweight='bold')
    
    plt.show()
    print("\n")    